In [29]:
import pickle
import time
import numpy as np
import pandas as pd
import math

import torch
import torchTools as tt

## Import beamline

Using 29ID/IEX

In [3]:
from beamline_29 import IEX

## Importing and testing GA package

Developed by Rebecca Aloisio during 2020 summer internship with XSD/BC.  Full repo and testing can be found here: https://github.com/Automated-Beamline-Endeavors/GA4beamlines

Locally kept in ../GA4beamlines and not quite a "package" hence the clunky import process.

In [7]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/beams/MWYMAN/sandbox/GA4beamlines/')

import ga4beamlines as ga4b

Imported!
I've been imported!


### Coordinating motor definitions

GA4beamlines uses a dictionary for the motors and beamline_29 had it's own class attribute for the positions and ranges

In [13]:
IEX.mask

array([[1, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 1, 0]])

In [16]:
IEX.sRange = IEX.sRange*0.4

In [18]:
motorNames = ['oe1x','oe1xrot','oe2x','oe2xrot','oe3xrot','oe4xrot','oe5y','oe5xrot','oe5yrot','oe6y','oe6xrot','oe6yrot',]

In [25]:
DOF_limits = IEX.sRange[IEX.mask == 1]

In [26]:
DOF_limits

array([[-8.160e+00,  8.160e+00],
       [-4.000e-03,  4.000e-03],
       [-9.160e+00,  9.160e+00],
       [-4.000e-03,  4.000e-03],
       [-8.000e-03,  8.000e-03],
       [-8.000e-03,  8.000e-03],
       [-1.680e+01,  1.680e+01],
       [-1.600e-01,  1.600e-01],
       [-1.288e+00,  1.288e+00],
       [-3.452e+01,  3.452e+01],
       [-5.520e-01,  5.520e-01],
       [-3.480e+00,  3.480e+00]])

In [33]:
DOF_sigma = [abs((lim[1]-lim[0])/5.) for lim in DOF_limits]

In [38]:
len(DOF_limits)

12

In [39]:
motors_29id = []
for name, limits, sigma in zip(motorNames, DOF_limits, DOF_sigma):
    motors_29id.append({'name':name, 'lo':min(limits), 'hi':max(limits), 'sigma':sigma})

In [40]:
motors_29id

[{'name': 'oe1x', 'lo': -8.16, 'hi': 8.16, 'sigma': 3.2640000000000002},
 {'name': 'oe1xrot', 'lo': -0.004, 'hi': 0.004, 'sigma': 0.0016},
 {'name': 'oe2x', 'lo': -9.16, 'hi': 9.16, 'sigma': 3.664},
 {'name': 'oe2xrot', 'lo': -0.004, 'hi': 0.004, 'sigma': 0.0016},
 {'name': 'oe3xrot', 'lo': -0.008, 'hi': 0.008, 'sigma': 0.0032},
 {'name': 'oe4xrot', 'lo': -0.008, 'hi': 0.008, 'sigma': 0.0032},
 {'name': 'oe5y', 'lo': -16.8, 'hi': 16.8, 'sigma': 6.720000000000001},
 {'name': 'oe5xrot',
  'lo': -0.16000000000000003,
  'hi': 0.16000000000000003,
  'sigma': 0.06400000000000002},
 {'name': 'oe5yrot',
  'lo': -1.2880000000000003,
  'hi': 1.2880000000000003,
  'sigma': 0.5152000000000001},
 {'name': 'oe6y', 'lo': -34.52, 'hi': 34.52, 'sigma': 13.808000000000002},
 {'name': 'oe6xrot',
  'lo': -0.5519999999999999,
  'hi': 0.5519999999999999,
  'sigma': 0.22079999999999997},
 {'name': 'oe6yrot', 'lo': -3.48, 'hi': 3.48, 'sigma': 1.392}]

### General GA setup

In [41]:
nGenerations = 10
critVal = 0.9 # TODO what's this for beamline? 
startPop = None

popSize = 100
nElite = 10
alpha = 0.75
s = 1.5

In [42]:
survivSelMode = ga4b.sMode[1]       # Genitor
parentSelMode = ga4b.pMode[0]       # ProbRank
crossOverMode = ga4b.cxMode[1]      # Simple
mutationMode = ga4b.mMode[1]        # Gaussian mutation
# fitnessMode = {"type": "Func", "name": funcnName} to be set in each section

## GA vs Shadow

First need to create fitness function based on shadow ouput

In [43]:
def shadowFitness(x, beamline):
    '''
        x                  : array of positions for moveable DOF
        beamline           : shadow beamline model
    '''    
    
    #TODO add error checking to make sure x is same shape as where beamline.mask == 1
    
    beamline.pos[beamline.mask == 1] = x #need to expand to handle masking as x only covers the DOF but pos includes unmoved DOF
    fitness = beamline.run()

    return fitness

In [47]:
fitnessMode = {"type": "Func", "name": shadowFitness, "args":IEX}

In [48]:
GA29id = ga4b.GA4Beamline(motors_29id, survivSelMode, parentSelMode, crossOverMode, mutationMode, fitnessMode)

In [49]:
GA29id.population

,oe1x,oe1xrot,oe2x,oe2xrot,oe3xrot,oe4xrot,oe5y,oe5xrot,oe5yrot,oe6y,oe6xrot,oe6yrot,fitness,ranking,probability
0,-8.030350,0.000956,-9.136891,0.003505,-0.000479,0.003785,10.101513,0.123039,0.777452,20.724196,0.327964,-2.555541,0.0,0,0.0
1,2.752769,0.001836,0.920625,0.000158,-0.001490,-0.002628,-4.632740,-0.113661,1.169595,0.681830,0.050726,0.197676,0.0,0,0.0
2,-4.904119,-0.002258,2.617507,-0.003225,0.007613,0.005034,9.524287,-0.082389,-0.008884,14.516457,0.427365,-0.994584,0.0,0,0.0
3,6.510549,0.001060,-1.893244,-0.003764,-0.007037,-0.001951,-3.227305,0.006425,-0.615829,21.536369,-0.536473,2.421171,0.0,0,0.0
4,7.416224,-0.002505,-7.107801,0.000300,0.002584,-0.001321,16.303213,-0.073944,1.101421,7.741947,-0.504010,-1.485260,0.0,0,0.0
5,1.325599,0.003067,-9.142148,0.003238,-0.003153,0.007377,-14.560673,0.020992,-0.929841,-22.485603,0.286890,-2.608384,0.0,0,0.0
6,0.284522,0.000003,-0.675018,0.002914,-0.004688,0.002656,11.070835,0.067893,1.256108,15.266385,0.499632,-0.212267,0.0,0,0.0
7,-1.552171,-0.002665,-4.092960,0.001470,0.007126,-0.006462,8.609323,0.146675,0.652178,18.032109,-0.427489,1.212431,0.0,0,0.0
8,-2.019454,-0.003104,3.115548,-0.001177,0.006355,0.001230,-3.961000,0.061102,1.025875,0.660709,0.512959,-2.549020,0.0,0,0.0
9,1.940412,-0.001233,-7.424761,-0.000318,0.003549,0.001183,-5.929511,-0.000477,-0.007235,-0.536544,0.284935,-2.979522,0.0,0,0.0


In [50]:
GA29id.FirstGeneration()

TypeError: shadowFitness() missing 1 required positional argument: 'beamline'

SyntaxError: can't use starred expression here (<ipython-input-52-e6063ee9fe5e>, line 4)

## GA vs Surrogate

First need to import scaling and model

In [ ]:
#import scaling
input_fn = 'IEX_100k_04w.pkl'

scaling_fn = input_fn.split('.')[0]+'_scaling.pkl'
with open(scaling_fn, 'rb') as f:
    scaling = pickle.load(f)

In [ ]:
#import model
nn_fn = input_fn.split('.')[0]+'_NN_results.pkl'
with open(nn_fn, 'rb') as f:
    result = pickle.load(f)

Need to create fitness function based on surrogate NN output

In [ ]:
def surrogateNNFitness(x, surrogate_model):
    '''
        x                  : array of positions that is the expected input for the surrogate model
        surrogate_model    : trained pyTorch model
    '''
    
    x_pd = pd.DataFrame(x, columns=['oe1x','oe1xrot','oe2x','oe2xrot','oe3xrot','oe4xrot','oe5y','oe5xrot','oe5yrot','oe6y','oe6xrot','oe6yrot',])
    xFeatures = scaling.transform(x_pd)
    xFeatures = pd.DataFrame(xFeatures, columns=x_pd.columns)

    x_tensor = torch.tensor(xFeatures.values) 
    
    model = surrogate_model
    device = torch.device('cpu')
    tt.to_device(model, device)

    fitness = model(tt.to_device(x_tensor.float(), device))
    
    return fitness
    

In [ ]:
surrogateNNFitness(test_pos, result[4]['model'])

In [ ]:
fitnessMode = {"type": "Func", "name": surrogateNNFitness}